In [1]:
# random forest classifier 
# hyper parameter 
# grid search cv 
# autoencoder? (oversampling된 데이터)

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import warnings
warnings.filterwarnings('ignore')
from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics   
from sklearn.ensemble import (AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier)
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import (AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier)
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold, cross_val_score

In [22]:
data = pd.read_csv("ins_fraud.csv")

In [109]:
models = []

models.append(("Random Forest Classifier: ", RandomForestClassifier()))
models.append(("Decision Tree Classifier: ", DecisionTreeClassifier()))
models.append(("AdaBoost Classifier: " , AdaBoostClassifier()))
models.append(("Gradient Boosting Classifier: ", GradientBoostingClassifier()))
models.append(("Logistic Regression: ", LogisticRegression()))
models.append(("Light GBM Classifier: ", LGBMClassifier()))

In [110]:
def results(X_train, y_train):
    results = []
    names = []

    for name, model in models:
        kfold = KFold(n_splits=5, random_state=13, shuffle=True)
        cv_results = cross_val_score(model, X_train, y_train,
                                    cv=kfold, scoring='accuracy')
        results.append(cv_results)
        names.append(name)
        model.fit(X_train, y_train)
        pred = model.predict(X_test)
        
        print(name)
        print("=======================================")
        print(f"cv_results mean: {cv_results.mean()}", 
              f"cv_results std: {cv_results.std()}")
        print('Acc : ', accuracy_score(y_test, pred))
        print(classification_report(y_test, pred))    
        print(confusion_matrix(y_test, pred))
        print(" ")

In [111]:
# ROC Curve
from sklearn.metrics import roc_curve
def draw_roc_curve(models, model_names, X_test, y_test):
    plt.figure(figsize=(10,10))
    for model in range(len(models)):
        pred = models[model].predict_proba(X_test)[:, 1]
        fpr, tpr, thresholds =  roc_curve(y_test, pred)
        plt.plot(fpr, tpr, label=model_names[model])
    plt.plot([0,1], [0,1], 'k--', label='random guess')
    plt.title('ROC')
    plt.legend()
    plt.grid()
    plt.show()

# UNDERSAMPLING

### RandomUnderSampler

In [112]:
# 맞춰야하는 테스ㅡㅌ 데이터의 fraud는 7개 
len(data[(data['c24']==1)&(data['fraud']==1)][["fraud"]])

7

In [113]:
train = data[data['c24'] == 0]
test = data[data['c24'] == 1]
X = data.drop(['fraud', 'c24'], axis=1)
y = data['fraud']
X_train = train.drop(['fraud', 'c24'], axis=1)
y_train = train['fraud']
X_test = test.drop(['fraud', 'c24'], axis=1)
y_test = test['fraud']

In [114]:
from imblearn.under_sampling import RandomUnderSampler 
rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X_train, y_train)

In [115]:
len(X_res), len(y_res)

(68, 68)

In [116]:
results(X_res, y_res)

Random Forest Classifier: 
cv_results mean: 0.5736263736263736 cv_results std: 0.12200276949232196
Acc :  0.45850688881768664
              precision    recall  f1-score   support

           0       1.00      0.46      0.63      3114
           1       0.00      0.57      0.00         7

    accuracy                           0.46      3121
   macro avg       0.50      0.51      0.32      3121
weighted avg       1.00      0.46      0.63      3121

[[1427 1687]
 [   3    4]]
 
Decision Tree Classifier: 
cv_results mean: 0.45714285714285713 cv_results std: 0.08909912879395379
Acc :  0.594360781800705
              precision    recall  f1-score   support

           0       1.00      0.59      0.75      3114
           1       0.00      0.43      0.00         7

    accuracy                           0.59      3121
   macro avg       0.50      0.51      0.37      3121
weighted avg       1.00      0.59      0.74      3121

[[1852 1262]
 [   4    3]]
 
AdaBoost Classifier: 
cv_results mean

In [118]:
# 하지만 데이터가 너무 적어서 유의미하다고 판단하기 어렵다.... 

### Tomek’s link method

In [119]:
train = data[data['c24'] == 0]
test = data[data['c24'] == 1]
X = data.drop(['fraud', 'c24'], axis=1)
y = data['fraud']
X_train = train.drop(['fraud', 'c24'], axis=1)
y_train = train['fraud']
X_test = test.drop(['fraud', 'c24'], axis=1)
y_test = test['fraud']

In [120]:
from imblearn.under_sampling import TomekLinks
tom = TomekLinks()
X_tom, y_tom = tom.fit_resample(X_train, y_train)

In [121]:
len(X_tom), len(y_tom)

(12861, 12861)

In [122]:
results(X_tom, y_tom)

Random Forest Classifier: 
cv_results mean: 0.997356324409664 cv_results std: 0.0007539908911836323
Acc :  0.9977571291252804
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3114
           1       0.00      0.00      0.00         7

    accuracy                           1.00      3121
   macro avg       0.50      0.50      0.50      3121
weighted avg       1.00      1.00      1.00      3121

[[3114    0]
 [   7    0]]
 
Decision Tree Classifier: 
cv_results mean: 0.9932353806939996 cv_results std: 0.0003106356347523787
Acc :  0.9900672861262416
              precision    recall  f1-score   support

           0       1.00      0.99      1.00      3114
           1       0.00      0.00      0.00         7

    accuracy                           0.99      3121
   macro avg       0.50      0.50      0.50      3121
weighted avg       1.00      0.99      0.99      3121

[[3090   24]
 [   7    0]]
 
AdaBoost Classifier: 
cv_results me

### Condensed Nearest Neighbour

In [123]:
train = data[data['c24'] == 0]
test = data[data['c24'] == 1]
X = data.drop(['fraud', 'c24'], axis=1)
y = data['fraud']
X_train = train.drop(['fraud', 'c24'], axis=1)
y_train = train['fraud']
X_test = test.drop(['fraud', 'c24'], axis=1)
y_test = test['fraud']

In [124]:
from imblearn.under_sampling import CondensedNearestNeighbour
cnn = CondensedNearestNeighbour(random_state=42)
X_cnn, y_cnn = cnn.fit_resample(X_train, y_train)

In [125]:
len(X_cnn), len(y_cnn)

(324, 324)

In [126]:
results(X_cnn, y_cnn)

Random Forest Classifier: 
cv_results mean: 0.8949999999999999 cv_results std: 0.018349016064425495
Acc :  0.9951938481256007
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3114
           1       0.10      0.14      0.12         7

    accuracy                           1.00      3121
   macro avg       0.55      0.57      0.56      3121
weighted avg       1.00      1.00      1.00      3121

[[3105    9]
 [   6    1]]
 
Decision Tree Classifier: 
cv_results mean: 0.8270673076923076 cv_results std: 0.02714350015365955
Acc :  0.8535725728933035
              precision    recall  f1-score   support

           0       1.00      0.86      0.92      3114
           1       0.00      0.00      0.00         7

    accuracy                           0.85      3121
   macro avg       0.50      0.43      0.46      3121
weighted avg       1.00      0.85      0.92      3121

[[2664  450]
 [   7    0]]
 
AdaBoost Classifier: 
cv_results mean

### One Sided Selection

In [127]:
train = data[data['c24'] == 0]
test = data[data['c24'] == 1]
X = data.drop(['fraud', 'c24'], axis=1)
y = data['fraud']
X_train = train.drop(['fraud', 'c24'], axis=1)
y_train = train['fraud']
X_test = test.drop(['fraud', 'c24'], axis=1)
y_test = test['fraud']

In [128]:
from imblearn.under_sampling import OneSidedSelection
oss = OneSidedSelection(random_state=0)
X_oss, y_oss = oss.fit_resample(X_train, y_train)

In [129]:
len(X_oss), len(y_oss)

(12802, 12802)

In [130]:
results(X_cnn, y_cnn)

Random Forest Classifier: 
cv_results mean: 0.8949999999999999 cv_results std: 0.018349016064425495
Acc :  0.9958346683755207
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3114
           1       0.00      0.00      0.00         7

    accuracy                           1.00      3121
   macro avg       0.50      0.50      0.50      3121
weighted avg       1.00      1.00      1.00      3121

[[3108    6]
 [   7    0]]
 
Decision Tree Classifier: 
cv_results mean: 0.8209134615384615 cv_results std: 0.023636054910157425
Acc :  0.8420378083947453
              precision    recall  f1-score   support

           0       1.00      0.84      0.91      3114
           1       0.00      0.14      0.00         7

    accuracy                           0.84      3121
   macro avg       0.50      0.49      0.46      3121
weighted avg       1.00      0.84      0.91      3121

[[2627  487]
 [   6    1]]
 
AdaBoost Classifier: 
cv_results mea

### Edited Nearest Neighbours

In [131]:
train = data[data['c24'] == 0]
test = data[data['c24'] == 1]
X = data.drop(['fraud', 'c24'], axis=1)
y = data['fraud']
X_train = train.drop(['fraud', 'c24'], axis=1)
y_train = train['fraud']
X_test = test.drop(['fraud', 'c24'], axis=1)
y_test = test['fraud']

In [132]:
from imblearn.under_sampling import EditedNearestNeighbours
enn = EditedNearestNeighbours(kind_sel="all", n_neighbors=5)
X_enn, y_enn = enn.fit_resample(X_train, y_train)

In [133]:
len(X_enn), len(y_enn)

(12726, 12726)

In [134]:
results(X_enn, y_enn)

Random Forest Classifier: 
cv_results mean: 0.9973283103662742 cv_results std: 0.0008013965347143231
Acc :  0.9977571291252804
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3114
           1       0.00      0.00      0.00         7

    accuracy                           1.00      3121
   macro avg       0.50      0.50      0.50      3121
weighted avg       1.00      1.00      1.00      3121

[[3114    0]
 [   7    0]]
 
Decision Tree Classifier: 
cv_results mean: 0.9926920459228004 cv_results std: 0.0012577689720476176
Acc :  0.9913489266260814
              precision    recall  f1-score   support

           0       1.00      0.99      1.00      3114
           1       0.00      0.00      0.00         7

    accuracy                           0.99      3121
   macro avg       0.50      0.50      0.50      3121
weighted avg       1.00      0.99      0.99      3121

[[3094   20]
 [   7    0]]
 
AdaBoost Classifier: 
cv_results m

### Neighbourhood Cleaning Rule

In [135]:
train = data[data['c24'] == 0]
test = data[data['c24'] == 1]
X = data.drop(['fraud', 'c24'], axis=1)
y = data['fraud']
X_train = train.drop(['fraud', 'c24'], axis=1)
y_train = train['fraud']
X_test = test.drop(['fraud', 'c24'], axis=1)
y_test = test['fraud']

In [136]:
from imblearn.under_sampling import NeighbourhoodCleaningRule
ncr = NeighbourhoodCleaningRule(kind_sel="all", n_neighbors=5)
X_ncr, y_ncr = ncr.fit_resample(X_train, y_train)

In [137]:
len(X_ncr), len(y_ncr)

(12709, 12709)

In [138]:
results(X_res, y_res)

Random Forest Classifier: 
cv_results mean: 0.6032967032967033 cv_results std: 0.0850069433222581
Acc :  0.48413969881448254
              precision    recall  f1-score   support

           0       1.00      0.48      0.65      3114
           1       0.00      0.43      0.00         7

    accuracy                           0.48      3121
   macro avg       0.50      0.46      0.33      3121
weighted avg       1.00      0.48      0.65      3121

[[1508 1606]
 [   4    3]]
 
Decision Tree Classifier: 
cv_results mean: 0.42637362637362636 cv_results std: 0.12200276949232194
Acc :  0.5414931111823134
              precision    recall  f1-score   support

           0       1.00      0.54      0.70      3114
           1       0.00      0.29      0.00         7

    accuracy                           0.54      3121
   macro avg       0.50      0.41      0.35      3121
weighted avg       0.99      0.54      0.70      3121

[[1688 1426]
 [   5    2]]
 
AdaBoost Classifier: 
cv_results mean

# Oversampling

### SMOTE

In [139]:
train = data[data['c24'] == 0]
test = data[data['c24'] == 1]
X = data.drop(['fraud', 'c24'], axis=1)
y = data['fraud']
X_train = train.drop(['fraud', 'c24'], axis=1)
y_train = train['fraud']
X_test = test.drop(['fraud', 'c24'], axis=1)
y_test = test['fraud']

In [140]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X_train, y_train)

In [141]:
len(X_smote), len(y_smote)

(25690, 25690)

In [142]:
results(X_smote, y_smote)

Random Forest Classifier: 
cv_results mean: 0.9979369404437526 cv_results std: 0.00036098164637976717
Acc :  0.9974367190003204
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3114
           1       0.00      0.00      0.00         7

    accuracy                           1.00      3121
   macro avg       0.50      0.50      0.50      3121
weighted avg       1.00      1.00      1.00      3121

[[3113    1]
 [   7    0]]
 
Decision Tree Classifier: 
cv_results mean: 0.9938497469832619 cv_results std: 0.0012284728562132468
Acc :  0.9868631848766422
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      3114
           1       0.00      0.00      0.00         7

    accuracy                           0.99      3121
   macro avg       0.50      0.49      0.50      3121
weighted avg       1.00      0.99      0.99      3121

[[3080   34]
 [   7    0]]
 
AdaBoost Classifier: 
cv_results 

### ADASYN(Adaptive Synthetic Sampling) 
#방법은 소수 클래스 데이터와 그 데이터에서 가장 가까운 k개의 소수 클래스 데이터 중 무작위로 선택된 데이터 사이의 
#직선상에 가상의 소수 클래스 데이터를 만드는 방법이다.

In [149]:
from imblearn.over_sampling import ADASYN

In [150]:
train = data[data['c24'] == 0]
test = data[data['c24'] == 1]
X = data.drop(['fraud', 'c24'], axis=1)
y = data['fraud']
X_train = train.drop(['fraud', 'c24'], axis=1)
y_train = train['fraud']
X_test = test.drop(['fraud', 'c24'], axis=1)
y_test = test['fraud']

In [151]:
from imblearn.over_sampling import ADASYN

adasyn = ADASYN(random_state=42)
X_adasyn, y_adasyn = adasyn.fit_resample(X_train, y_train)

In [152]:
len(X_adasyn), len(y_adasyn)

(25697, 25697)

In [153]:
results(X_adasyn, y_adasyn)

Random Forest Classifier: 
cv_results mean: 0.9978597177455075 cv_results std: 0.0005362179831137537
Acc :  0.9967958987504005
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3114
           1       0.00      0.00      0.00         7

    accuracy                           1.00      3121
   macro avg       0.50      0.50      0.50      3121
weighted avg       1.00      1.00      1.00      3121

[[3111    3]
 [   7    0]]
 
Decision Tree Classifier: 
cv_results mean: 0.9938903767103323 cv_results std: 0.0010767483911312873
Acc :  0.9852611342518424
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      3114
           1       0.00      0.00      0.00         7

    accuracy                           0.99      3121
   macro avg       0.50      0.49      0.50      3121
weighted avg       1.00      0.99      0.99      3121

[[3075   39]
 [   7    0]]
 
AdaBoost Classifier: 
cv_results m

## 복합 샘플링

- SMOTEENN: SMOTE + ENN
- SMOTETomek: SMOTE + Tomek

In [154]:
from imblearn.over_sampling import *

### SMOTETomek

In [155]:
train = data[data['c24'] == 0]
test = data[data['c24'] == 1]
X = data.drop(['fraud', 'c24'], axis=1)
y = data['fraud']
X_train = train.drop(['fraud', 'c24'], axis=1)
y_train = train['fraud']
X_test = test.drop(['fraud', 'c24'], axis=1)
y_test = test['fraud']

In [156]:
from imblearn.combine import *
stt = SMOTETomek(random_state=4)
X_stt, y_stt = stt.fit_resample(X_train, y_train)

In [157]:
len(X_stt), len(y_stt)

(22476, 22476)

In [158]:
results(X_stt, y_stt)

Random Forest Classifier: 
cv_results mean: 0.9979534199723694 cv_results std: 0.0006506620966773755
Acc :  0.9961550785004806
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3114
           1       0.00      0.00      0.00         7

    accuracy                           1.00      3121
   macro avg       0.50      0.50      0.50      3121
weighted avg       1.00      1.00      1.00      3121

[[3109    5]
 [   7    0]]
 
Decision Tree Classifier: 
cv_results mean: 0.9923474184443766 cv_results std: 0.001576733048242416
Acc :  0.9855815443768023
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      3114
           1       0.00      0.00      0.00         7

    accuracy                           0.99      3121
   macro avg       0.50      0.49      0.50      3121
weighted avg       1.00      0.99      0.99      3121

[[3076   38]
 [   7    0]]
 
AdaBoost Classifier: 
cv_results me

### SMOTEENN

In [159]:
train = data[data['c24'] == 0]
test = data[data['c24'] == 1]
X = data.drop(['fraud', 'c24'], axis=1)
y = data['fraud']
X_train = train.drop(['fraud', 'c24'], axis=1)
y_train = train['fraud']
X_test = test.drop(['fraud', 'c24'], axis=1)
y_test = test['fraud']

In [160]:
from imblearn.combine import SMOTEENN 

sme = SMOTEENN(random_state=42)
X_sme, y_sme = sme.fit_resample(X_train, y_train)

In [161]:
len(X_sme), len(y_sme)

(15812, 15812)

In [162]:
results(X_sme, y_sme)

Random Forest Classifier: 
cv_results mean: 0.9975967778930283 cv_results std: 0.0008146355849124103
Acc :  0.9958346683755207
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3114
           1       0.00      0.00      0.00         7

    accuracy                           1.00      3121
   macro avg       0.50      0.50      0.50      3121
weighted avg       1.00      1.00      1.00      3121

[[3108    6]
 [   7    0]]
 
Decision Tree Classifier: 
cv_results mean: 0.9926639114540496 cv_results std: 0.001474324129021288
Acc :  0.9762896507529638
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      3114
           1       0.00      0.00      0.00         7

    accuracy                           0.98      3121
   macro avg       0.50      0.49      0.49      3121
weighted avg       1.00      0.98      0.99      3121

[[3047   67]
 [   7    0]]
 
AdaBoost Classifier: 
cv_results me